# 4. Análise de centralidade e comunidades no grafo de autoria



## 4.1. Preparação



### 4.1.1. Imports

In [1]:

import os

from pathlib import Path
from collections import defaultdict
import duckdb
import numpy as np
import pandas as pd
import igraph as ig
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

from event import Event

from dotenv import load_dotenv

load_dotenv()

PROJECT_DIR = Path("~/tramita").expanduser()
DB_PATH = PROJECT_DIR / os.getenv("SILVER_DUCKDB_PATH", "")
OUT_DIR = PROJECT_DIR / "data" / "gold"
OUT_DIR.mkdir(exist_ok=True)
ACCESS_DIR = OUT_DIR / "accessory_data"
ACCESS_DIR.mkdir(exist_ok=True)

NODES_PATH_PARQUET = OUT_DIR / "nodes.parquet"
EDGES_PATH_PARQUET = OUT_DIR / "edges.parquet"
NODES_PATH_CSV = OUT_DIR / "nodes.csv"
EDGES_PATH_CSV = OUT_DIR / "edges.csv"

# with duckdb.connect(DB_PATH, read_only=True) as con:

### 4.1.2. Funções auxiliares

In [2]:
def merge_nodes(nodes_df: pd.DataFrame, edges_df: pd.DataFrame, taglist: list[str]) -> None:
    """Merges the indicated nodes and consolidates the respective edges."""
    surviving_tag = taglist[0]
    for removed_tag in taglist[1:]:
        edges_df.loc[edges_df['source'] == removed_tag, 'source'] = surviving_tag
        edges_df.loc[edges_df['target'] == removed_tag, 'target'] = surviving_tag

    nodes_df.drop(index=taglist[1:], inplace=True)
    

def prune_graph(
    node_df: pd.DataFrame,
    edge_df: pd.DataFrame,
    tag_col: str = "name",
    from_col: str = "from",
    to_col: str = "to",
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Removes any orphan nodes, then edges, until every node has degree at least 1."""
    def _prune_nodes(node_df: pd.DataFrame, edge_df: pd.DataFrame) -> pd.DataFrame:
        return node_df[
            (node_df[tag_col].isin(edge_df[from_col]))
            | (node_df[tag_col].isin(edge_df[to_col]))
        ]
        
        
    def _prune_edges(node_df: pd.DataFrame, edge_df: pd.DataFrame) -> pd.DataFrame:
        return edge_df[
            (edge_df[from_col].isin(node_df[tag_col]))
            & (edge_df[to_col].isin(node_df[tag_col]))
        ]

    while True:
        n = len(node_df)
        m = len(edge_df)
        node_df = _prune_nodes(node_df, edge_df)
        edge_df = _prune_edges(node_df, edge_df)
        if n == len(node_df) and m == len(edge_df):
            return node_df, edge_df
        
def build_graph(node_df: pd.DataFrame, edge_df: pd.DataFrame) -> ig.Graph:
    """
    Builds an igraph.Graph from a node and edge list.
    The node DataFrame must have a 'name' column with the unique identifiers.
    The edge DataFrame must have 'from' and 'to' columns.
    Any other columns will be absorbed as attributes.
    """
    edge_tuples = list(zip(edge_df['from'], edge_df['to']))
    g = ig.Graph.TupleList(
        edge_tuples,
        directed=False,
        vertex_name_attr="name",
        weights=True,
    )
    for col in node_df.columns:
        if col != "name":
            g.vs[col] = node_df.set_index("name").loc[g.vs['name'], col].tolist()

    for col in edge_df.columns:
        if col not in ("from", "to"):
            g.es[col] = edge_df[col].tolist()
    
    return g


## 4.2. Qualificação de arestas de autoria de acordo com o estágio de tramitação no Congresso

### 4.2.1. Carga dos dados

In [3]:
event_df = pd.read_pickle(ACCESS_DIR / "full_event_df.pkl")
edges_df = pd.read_parquet(EDGES_PATH_PARQUET)
nodes_df = pd.read_parquet(NODES_PATH_PARQUET)

### 4.2.1. Definição da casa inicial de tramitação de cada proposição

Proposições protocoladas por senadores iniciam a tramitação no Senado; as demais, na Câmara.

In [4]:
# agregamos as autorias aos eventos

event_labeled_df = event_df.join(
    edges_df[edges_df['etype'] == 'autoria'].set_index('target')[['source']], on="prop_tag"
).rename(columns={'source': 'auth_camara_tag'}).join(
    edges_df[edges_df['etype'] == 'autoria'].set_index('target')[['source']], on="proc_tag"
).rename(columns={'source': 'auth_senado_tag'}).join(
    nodes_df.set_index('tag')[['label']], on="auth_camara_tag", rsuffix="_auth"
).rename(columns={'label_auth': 'auth_camara_label'}).join(
    nodes_df.set_index('tag')[['label']], on="auth_senado_tag", rsuffix="_auth"
).rename(columns={'label_auth': 'auth_senado_label'})
event_labeled_df

,label,event_ts,event,event_loc,casa,prop_tag,proc_tag,auth_camara_tag,auth_senado_tag,auth_camara_label,auth_senado_label
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
...,...,...,...,...,...,...,...,...,...,...,...
114703,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara,CP:2354532,NaN,CD:204355,NaN,JOSIAS MARIO DA VITORIA,NaN
114704,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara,CP:2250966,NaN,CD:141531,NaN,RODRIGO BATISTA DE CASTRO,NaN
114705,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara,CP:2284649,NaN,CD:204569,NaN,PABLO OLIVA SOUZA,NaN
114706,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara,CP:2322707,NaN,CD:178881,NaN,ALUISIO GUIMARAES MENDES FILHO,NaN


In [5]:
# como agora temos uma fileira para cada autor, vamos manter apenas o primeiro (só precisamos saber de onde sai cada projeto)

event_labeled_df = event_labeled_df[event_labeled_df[['label', 'event_ts', 'event', 'event_loc']].ne(
    event_labeled_df[['label', 'event_ts', 'event', 'event_loc']].shift()
).any(axis=1)].reset_index(drop=True)
event_labeled_df

,label,event_ts,event,event_loc,casa,prop_tag,proc_tag,auth_camara_tag,auth_senado_tag,auth_camara_label,auth_senado_label
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA
...,...,...,...,...,...,...,...,...,...,...,...
114806,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara,CP:2354532,NaN,CD:204355,NaN,JOSIAS MARIO DA VITORIA,NaN
114807,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara,CP:2250966,NaN,CD:141531,NaN,RODRIGO BATISTA DE CASTRO,NaN
114808,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara,CP:2284649,NaN,CD:204569,NaN,PABLO OLIVA SOUZA,NaN
114809,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara,CP:2322707,NaN,CD:178881,NaN,ALUISIO GUIMARAES MENDES FILHO,NaN


In [6]:
for col in event_labeled_df.columns:
    if col.endswith("_tag"):
        event_labeled_df[col] = event_labeled_df[col].str.strip().str.upper()

event_labeled_df = event_labeled_df.fillna('')

Processo vetorizado para definição da origem

In [7]:
from_camara = (
    event_labeled_df['auth_camara_tag'].str.startswith("CD:")
    | (event_labeled_df['auth_senado_tag'] == "SE:2")
    | (event_labeled_df['auth_camara_tag'].isin([
        "CO:5438",  # Comissão de Legislação Participativa
        "CO:2003",  # CCJC
        "CO:539426",  # CPI da Americanas
    ]))
    | (event_labeled_df['auth_camara_label'].str.startswith("Comissão Mista da MPV"))  # Todas começam na Câmara
)
from_senado = (
    event_labeled_df['auth_senado_tag'].str.startswith("SS:")
    | (event_labeled_df['auth_camara_tag'].isin([
        "CO:78",  # Senado
        "CO:79",  # Comissão mista (na verdade o autor é o Sen. Jorginho Mello)
    ]))
    | (event_labeled_df['auth_senado_tag'].isin([
        "SE:7352398",  # CPI da Pandemia
        "SE:3947422",  # Comissão de direitos humanos do Senado 
    ]))
)
from_externo = ~(from_camara | from_senado)

event_labeled_df['origem'] = np.select(
    [from_camara, from_senado, from_externo],
    ['camara', 'senado', 'externo'],
    default='unknown'
)
event_labeled_df

,label,event_ts,event,event_loc,casa,prop_tag,proc_tag,auth_camara_tag,auth_senado_tag,auth_camara_label,auth_senado_label,origem
0,MPV 1000/2020,2020-09-03 00:00:00,Event.APRESENTADO,SF,senado,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA,externo
1,MPV 1000/2020,2020-09-03 10:57:00,Event.APRESENTADO,EXEC,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA,externo
2,MPV 1000/2020,2020-09-09 00:00:00,Event.RECEBIDO_COMISSAO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA,externo
3,MPV 1000/2020,2020-09-09 00:00:00,Event.APRESENTADO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA,externo
4,MPV 1000/2020,2020-09-10 12:09:00,Event.DISTRIBUIDO,MESA,camara,CP:2262062,SP:7979012,CO:253,SE:55126,PODER EXECUTIVO,PRESIDÊNCIA DA REPÚBLICA,externo
...,...,...,...,...,...,...,...,...,...,...,...,...
114806,PLV 8/2023,2023-03-30 17:23:00,Event.APRESENTADO,MPV115222,camara,CP:2354532,,CD:204355,,JOSIAS MARIO DA VITORIA,,camara
114807,PLV 9/2020,2020-04-29 22:15:00,Event.APRESENTADO,PLEN,camara,CP:2250966,,CD:141531,,RODRIGO BATISTA DE CASTRO,,camara
114808,PLV 9/2021,2021-05-25 19:23:00,Event.APRESENTADO,PLEN,camara,CP:2284649,,CD:204569,,PABLO OLIVA SOUZA,,camara
114809,PLV 9/2022,2022-05-11 17:23:00,Event.APRESENTADO,MPV108021,camara,CP:2322707,,CD:178881,,ALUISIO GUIMARAES MENDES FILHO,,camara


### 4.2.2. Atribuição de notas referentes ao progresso de proposições

Critérios para score de proposições:

(considerando que o caminho pode ser Câmara -> Senado ou Senado -> Câmara)

* Foi protocolada na primeira casa mas não chegou a comissão ou plenário: 0.0
* Chegou a comissão ou plenário na primeira casa: 0.25
* Aprovada na primeira casa: 0.5
* Chegou a comissão ou plenário na segunda casa: 0.75
* Aprovada na segunda casa (ou seja, remetida a sanção ou promulgação): 1.0

In [8]:
df = event_labeled_df
by_label = df.groupby('label', sort=False)
origem = by_label['origem'].first()
origem

label
MPV 1000/2020    externo
MPV 1001/2020    externo
MPV 1002/2020    externo
MPV 1003/2020    externo
MPV 1004/2020    externo
                  ...   
PLV 8/2023        camara
PLV 9/2020        camara
PLV 9/2021        camara
PLV 9/2022        camara
PLV 9/2023       externo
Name: origem, Length: 28081, dtype: object

In [9]:
is_mpv_pl = df['label'].str.startswith(('MPV','PL'))
is_pec = df['label'].str.startswith('PEC')
has_sancao = df['event'].eq(Event.REMETIDO_A_SANCAO).groupby(df['label']).any()
has_promulg = df['event'].eq(Event.REMETIDO_A_PROMULGACAO).groupby(df['label']).any()

In [10]:
presence = (
    df.assign(present=True).pivot_table(
        index='label',
        columns=['casa','event'],
        values='present',
        aggfunc='any',
        fill_value=False
    )
)
presence

casa                     camara                                            \
event         Event.APRESENTADO Event.DISTRIBUIDO Event.RECEBIDO_COMISSAO   
label                                                                       
MPV 1000/2020              True              True                    True   
MPV 1001/2020              True              True                    True   
MPV 1002/2020              True              True                    True   
MPV 1003/2020              True              True                    True   
MPV 1004/2020              True              True                    True   
...                         ...               ...                     ...   
PLV 8/2023                 True             False                   False   
PLV 9/2020                 True             False                   False   
PLV 9/2021                 True             False                   False   
PLV 9/2022                 True             False                   False   
PLV 9/2023                 True             False                   False   

casa                                                                          \
event         Event.DESIGNADO_RELATOR_COMISSAO Event.RETIRADO_PAUTA_COMISSAO   
label                                                                          
MPV 1000/2020                            False                         False   
MPV 1001/2020                            False                         False   
MPV 1002/2020                            False                         False   
MPV 1003/2020                            False                         False   
MPV 1004/2020                            False                         False   
...                                        ...                           ...   
PLV 8/2023                               False                         False   
PLV 9/2020                               False                         False   
PLV 9/2021                               False                         False   
PLV 9/2022                               False                         False   
PLV 9/2023                               False                         False   

casa                                                                    \
event         Event.APROVADA_URGENCIA Event.DESIGNADO_RELATOR_PLENARIO   
label                                                                    
MPV 1000/2020                   False                             True   
MPV 1001/2020                   False                            False   
MPV 1002/2020                   False                            False   
MPV 1003/2020                   False                             True   
MPV 1004/2020                   False                             True   
...                               ...                              ...   
PLV 8/2023                      False                            False   
PLV 9/2020                      False                            False   
PLV 9/2021                      False                            False   
PLV 9/2022                      False                            False   
PLV 9/2023                      False                            False   

casa                                                            \
event         Event.REMETIDO_AO_SENADO Event.REMETIDO_A_SANCAO   
label                                                            
MPV 1000/2020                    False                   False   
MPV 1001/2020                    False                   False   
MPV 1002/2020                    False                   False   
MPV 1003/2020                     True                   False   
MPV 1004/2020                     True                   False   
...                                ...                     ...   
PLV 8/2023                       False                   False   
PLV 9/2020                       False                   False   
PLV 9/2021                       Fals

In [11]:
# função auxiliar para retornar a coluna de presença de um evento numa casa
def P(house: str, event: Event) -> pd.Series:
    col = (house, event)
    return (
        presence[col]
        if col in presence.columns
        else pd.Series(False, index=presence.index)
    )

In [12]:
score_camara_externo = np.select(
    [
        (is_mpv_pl.groupby(df['label']).any() & has_sancao) |
        (is_pec.groupby(df['label']).any() & has_promulg),  # 1.0
        P('senado', Event.RECEBIDO_COMISSAO) | P('senado', Event.APROVADA_URGENCIA) | P('senado', Event.DESIGNADO_RELATOR_PLENARIO),  # 0.75
        P('camara', Event.APROVADO_PLENARIO),  # 0.5
        P('camara', Event.RECEBIDO_COMISSAO) | P('camara', Event.APROVADA_URGENCIA) | P('camara', Event.DESIGNADO_RELATOR_PLENARIO),  # 0.25
    ],
    [1.0, 0.75, 0.50, 0.25],
    default=0.0
)

In [13]:
score_senado = np.select(
    [
        (is_mpv_pl.groupby(df['label']).any() & has_sancao) |
        (is_pec.groupby(df['label']).any() & has_promulg),  # 1.0
        P('camara', Event.RECEBIDO_COMISSAO) | P('camara', Event.APROVADA_URGENCIA) | P('camara', Event.DESIGNADO_RELATOR_PLENARIO),  # 0.75
        P('senado', Event.APROVADO_PLENARIO),  # 0.5
        P('senado', Event.RECEBIDO_COMISSAO) | P('senado', Event.APROVADA_URGENCIA) | P('senado', Event.DESIGNADO_RELATOR_PLENARIO),  # 0.25
    ],
    [1.0, 0.75, 0.50, 0.25],
    default=0.0
)

In [14]:
scores = pd.DataFrame({
    'origem': origem,
    'score_camara_externo': score_camara_externo,
    'score_senado': score_senado,
})
scores['score'] = np.where(
    scores['origem'].isin(['camara','externo']),
    scores['score_camara_externo'],
    np.where(scores['origem'].eq('senado'), scores['score_senado'], np.nan)
)

In [15]:
labels_and_scores: list[dict] = (
    scores['score']
    .rename('score')
    .to_frame()
    .reset_index(names='label')
    .to_dict('records')
)

In [16]:
# Proposições por rótulo e score
labels_and_scores_df = pd.DataFrame(labels_and_scores).set_index('label', drop=True)
labels_and_scores_df

,score
label,
MPV 1000/2020,0.25
MPV 1001/2020,0.25
MPV 1002/2020,0.25
MPV 1003/2020,0.75
MPV 1004/2020,0.75
...,...
PLV 8/2023,0.00
PLV 9/2020,0.00
PLV 9/2021,0.00


In [17]:
# Agregamos os scores à tabela de vértices
nodes_scored_df = nodes_df.join(labels_and_scores_df, on="label")
nodes_scored_df = nodes_scored_df.set_index('tag', drop=True)
nodes_scored_df.loc[nodes_scored_df['type'].isin(['Proposicao', 'Processo']) & nodes_scored_df['score'].isna(), 'score'] = 0.0
nodes_scored_df

,label,partido,type,score
tag,,,,
CP:2187087,PL 5029/2019,None,Proposicao,1.00
CP:2190408,PL 2/2019,None,Proposicao,0.25
CP:2190417,PL 10/2019,None,Proposicao,0.25
CP:2190423,PL 15/2019,None,Proposicao,0.00
CP:2190450,PL 21/2019,None,Proposicao,0.25
...,...,...,...,...
SE:55143,TRIBUNAL DE JUSTIÇA DO DISTRITO FEDERAL E TERR...,None,Ente,NaN
SE:9999990,SUPERIOR TRIBUNAL DE JUSTIÇA,None,Ente,NaN
SE:9999991,PROCURADORIA-GERAL DA REPÚBLICA,None,Ente,NaN


## 4.3. Preparação do grafo para análise

### 4.3.1. Fusão de arestas de correspondência

Na etapa anterior, definimos arestas do tipo 'correspondência' para proposições, parlamentares, ou entidades que eram representados por vértices múltiplos. Aqui vamos fundir esses vértices e eliminar as arestas de correspondência.

In [18]:
# Primeiro determinamos os componentes conexos de correspondência para fusão

ccs: dict[str, set] = defaultdict(set)

for index, row in edges_df[edges_df['etype'].eq('correspondencia')].iterrows():
    src = row['source']
    tgt = row['target']
    src_set = ccs[src]
    tgt_set = ccs[tgt]
    new_set = {src, tgt}
    if not src_set and not tgt_set:
        ccs[src] = new_set
        ccs[tgt] = new_set
    elif not src_set:
        tgt_set.update(new_set)    
        ccs[src] = tgt_set
    else:  # no tgt_set
        src_set.update(new_set)
        ccs[tgt] = src_set
    
unique_ccs = {frozenset(s) for s in ccs.values()}

In [19]:
# Agora fazemos a fusão, eliminando as arestas de correspondência

nodes_to_merge = [sorted(s) for s in unique_ccs]

edges_auth_df = edges_df[edges_df['etype'].eq('autoria')].drop(['etype'], axis=1).copy()


for taglist in nodes_to_merge:
    merge_nodes(nodes_scored_df, edges_auth_df, taglist)

### 4.3.2. Atribuição do score às arestas de autoria e preparação dos DataFrames para ingestão no iGraph

In [20]:
edges_weighted_df = edges_auth_df.join(nodes_scored_df[['score']], on="target").rename(columns={'score': 'weight'})

edge_df = edges_weighted_df.rename(columns={'source': 'from', 'target': 'to'})
node_df = nodes_scored_df.reset_index().rename(columns={'tag': 'name'})

node_df, edge_df = prune_graph(node_df, edge_df)

In [21]:
# Finalmente, a partir deste ponto não existe mais diferença entre um órgão ou ente (nomes diferentes para a mesma coisa)
node_df.loc[node_df['type'] == "Ente", 'type'] = 'Orgao'

In [22]:
node_df.value_counts('type')

type
Proposicao    27046
Processo       1113
Deputado        916
Senador         114
Orgao            46
Name: count, dtype: int64

### 4.3.3. Colunas auxiliares

In [23]:
# definimos um tipo genérico para facilitar análise do grafo bipartite

type_to_bigtype = {
    'Proposicao': 'bill',
    'Processo': 'bill',
    'Orgao': 'author',
    'Deputado': 'author',
    'Senador': 'author',
}

node_df['bigtype'] = node_df['type'].map(type_to_bigtype)
node_df.value_counts('bigtype')

bigtype
bill      28159
author     1076
Name: count, dtype: int64

In [24]:
# Definimos uma coluna para o tipo de projeto de lei (PL, PLP, PEC...)
node_df['billtype'] = node_df.apply(
    lambda row: row['label'].split()[0] if row['bigtype'] == 'bill' else None,
    axis=1 
)
node_df

,name,label,partido,type,score,bigtype,billtype
0,CP:2187087,PL 5029/2019,None,Proposicao,1.00,bill,PL
1,CP:2190408,PL 2/2019,None,Proposicao,0.25,bill,PL
2,CP:2190417,PL 10/2019,None,Proposicao,0.25,bill,PL
3,CP:2190423,PL 15/2019,None,Proposicao,0.00,bill,PL
4,CP:2190450,PL 21/2019,None,Proposicao,0.25,bill,PL
...,...,...,...,...,...,...,...
29230,SE:7352398,CPI DA PANDEMIA,None,Orgao,NaN,author,None
29231,SE:55226,COMISSÃO DIRETORA,None,Orgao,NaN,author,None
29232,SE:3947422,COMISSÃO DE DIREITOS HUMANOS E LEGISLAÇÃO PART...,None,Orgao,NaN,author,None
29233,SE:3927825,COMISSÃO DE MEIO AMBIENTE,None,Orgao,NaN,author,None


In [25]:
# Separamos o partido da UF dos parlamentares
node_df['partido'] = node_df['partido'].str.replace("S/Partido", "Sem Partido")


In [26]:
node_df[['cod_partido', 'uf']] = node_df[node_df['partido'].notna()]['partido'].str.split("/", expand=True)

# 4.4. Análises

### 4.4.1. Análise dos componentes conexos

In [27]:
g = build_graph(node_df, edge_df)

ccs = g.connected_components("weak")


node_cc_df = node_df.copy().set_index('name', drop=True)
node_cc_df['cc_index'] = -1
for i, cc in enumerate(ccs):
    node_cc_df.loc[g.vs[cc]['name'], 'cc_index'] = i
cc_summary_df = node_cc_df.groupby('cc_index').agg(
    size=('cc_index', 'size'),
    type_counts=('type', lambda x: x.value_counts().to_dict()),
    bigtype_counts=('bigtype', lambda x: x.value_counts().to_dict())
).sort_values('size', ascending=False)
cc_summary_df.insert(1, 'fraction', cc_summary_df['size'] / len(node_cc_df))
cc_summary_df

,size,fraction,type_counts,bigtype_counts
cc_index,,,,
0,27680,0.946810,"{'Proposicao': 25596, 'Processo': 1113, 'Deput...","{'bill': 26709, 'author': 971}"
5,757,0.025894,"{'Proposicao': 755, 'Orgao': 2}","{'bill': 755, 'author': 2}"
6,60,0.002052,"{'Proposicao': 59, 'Deputado': 1}","{'bill': 59, 'author': 1}"
62,45,0.001539,"{'Proposicao': 44, 'Deputado': 1}","{'bill': 44, 'author': 1}"
13,42,0.001437,"{'Proposicao': 41, 'Orgao': 1}","{'bill': 41, 'author': 1}"
...,...,...,...,...
81,2,0.000068,"{'Proposicao': 1, 'Deputado': 1}","{'bill': 1, 'author': 1}"
70,2,0.000068,"{'Proposicao': 1, 'Orgao': 1}","{'bill': 1, 'author': 1}"
93,2,0.000068,"{'Proposicao': 1, 'Deputado': 1}","{'bill': 1, 'author': 1}"


Vemos algumas coisas interessantes aí:

* Um subgrafo conexo majoritário (94.7% dos vértices) ligando deputados, senadores, órgãos e proposições
* Vários subgrafos pequenos ligando poucos deputados ou órgãos e poucas proposições.

Vamos primeiro olhar para alguns grafos isolados.

**Projetos do Poder Executivo**

In [28]:
CC_INDEX_EXEC = 5
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_EXEC))&(node_cc_df['bigtype'].eq('author'))]

,label,partido,type,score,bigtype,billtype,cod_partido,uf,cc_index
name,,,,,,,,,
CO:253,PODER EXECUTIVO,None,Orgao,NaN,author,None,NaN,NaN,5
SE:7352682,FORUM NACIONAL DE COMITÊS HIDROGRÁFICAS BRASIL,None,Orgao,NaN,author,None,NaN,NaN,5


In [29]:
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_EXEC))&(node_cc_df['bigtype'].eq('bill'))].value_counts('billtype')

billtype
MPV    400
PLN    210
PL     123
PLP     18
PEC      4
Name: count, dtype: int64

In [30]:
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_EXEC))&(node_cc_df['bigtype'].eq('bill'))].value_counts('score')

score
1.00    311
0.75    206
0.25    137
0.00     77
0.50     24
Name: count, dtype: int64

Nenhuma surpresa aí. Medidas Provisórias são atribuição privativa do Poder Executivo e projetos do Executivo tendem a ser mais bem-sucedidos.

Outro subgrafo interessante é o da Comissão de Legislação Participativa

In [31]:
CC_INDEX_CLP = 13

In [32]:
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_CLP))&(node_cc_df['bigtype'].eq('author'))]

,label,partido,type,score,bigtype,billtype,cod_partido,uf,cc_index
name,,,,,,,,,
CO:5438,COMISSÃO DE LEGISLAÇÃO PARTICIPATIVA,None,Orgao,NaN,author,None,NaN,NaN,13


In [33]:
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_CLP))&(node_cc_df['bigtype'].eq('bill'))].value_counts('billtype')

billtype
PL     39
PLP     2
Name: count, dtype: int64

In [34]:
node_cc_df[(node_cc_df['cc_index'].eq(CC_INDEX_CLP))&(node_cc_df['bigtype'].eq('bill'))].value_counts('score')

score
0.25    37
0.00     4
Name: count, dtype: int64

Nenhuma proposição proposta pela comissão passou do plenário da Câmara.

Vejamos quem são os parlamentares campeões em propor projetos de lei sem colaborar com mais ninguém.

In [35]:
def filter_isolates(tcounts: dict):
    return len(tcounts) == 2 and (
        tcounts.get('Deputado', 0) == 1
        or tcounts.get('Senador', 0) == 1
    )
    


isolates_df = node_cc_df[
    (node_cc_df['cc_index'].isin(cc_summary_df[cc_summary_df['type_counts'].apply(filter_isolates)].index))
    & (node_cc_df['bigtype'].eq('author'))
].join(edge_df.groupby('from').agg({'weight': 'mean', 'to': 'count'}), how="inner")[[
    'label', 'partido', 'type', 'weight', 'to',
]].rename(columns={'weight': 'desempenho_medio', 'to': 'n_proposicoes'}).sort_values('n_proposicoes', ascending=False)
isolates_df['desempenho_medio'] = round(isolates_df['desempenho_medio'], 2)
isolates_df


,label,partido,type,desempenho_medio,n_proposicoes
CD:204397,EMERSON MIGUEL PETRIV,PROS/PR,Deputado,0.20,59
CD:220653,FÁBIO EDUARDO DE OLIVEIRA TERUEL,MDB/SP,Deputado,0.21,44
CD:122195,LEONARDO DE MELO GADELHA,PODE/PB,Deputado,0.18,32
CD:213856,FRANCISCO DEUZINHO DE OLIVEIRA FILHO,PROS/CE,Deputado,0.22,30
CD:204564,ANDERSON MACHADO DE JESUS,DEM/BA,Deputado,0.21,29
...,...,...,...,...,...
CD:213854,AGRIPINO RODRIGUES GOMES MAGALHÃES,UNIÃO/CE,Deputado,0.25,1
CD:74570,JUTAHY MAGALHÃES JÚNIOR,PSDB/BA,Deputado,0.25,1
CD:74124,MARÇAL GONÇALVES LEITE FILHO,PMDB/MS,Deputado,0.25,1
CD:105534,ALEXANDRE BRITO DE FIGUEIREDO,MDB/SE,Deputado,0.25,1


Vamos extrair alguns dados do grande grupo conexo agora.

In [36]:
node_main_df = node_cc_df[node_cc_df['cc_index'].eq(0)].drop('cc_index', axis=1).copy()
node_main_df

,label,partido,type,score,bigtype,billtype,cod_partido,uf
name,,,,,,,,
CP:2187087,PL 5029/2019,None,Proposicao,1.00,bill,PL,NaN,NaN
CP:2190408,PL 2/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
CP:2190417,PL 10/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
CP:2190423,PL 15/2019,None,Proposicao,0.00,bill,PL,NaN,NaN
CP:2190450,PL 21/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
...,...,...,...,...,...,...,...,...
SS:6295,CARLOS HENRIQUE BAQUETA FÁVARO,PSD/MT,Senador,NaN,author,None,PSD,MT
SE:7352398,CPI DA PANDEMIA,None,Orgao,NaN,author,None,NaN,NaN
SE:55226,COMISSÃO DIRETORA,None,Orgao,NaN,author,None,NaN,NaN


In [37]:
node_main_df, edge_main_df = prune_graph(node_main_df.reset_index(), edge_df)

In [38]:
node_main_df.shape, edge_main_df.shape

((27680, 9), (47755, 3))

In [39]:
# Para facilitar, vamos gerar um igraph.Graph só deste componente.

main_g = build_graph(node_main_df, edge_main_df)

In [40]:
len(main_g.connected_components())

1

Até o momento, o ponto mais interessante é que praticamente todo o Congresso está interligado em termos de autoria de projetos, especialmente tomando em consideração a polarização política característica do período de tempo abrangido. Vamos ver como isso acontece.

Primeiro vamos analisar as comunidades, sem levar em consideração os pesos das arestas por enquanto.

In [47]:
comms_leiden_mod = main_g.community_leiden(objective_function="modularity")

In [51]:
node_main_df

,name,label,partido,type,score,bigtype,billtype,cod_partido,uf
0,CP:2187087,PL 5029/2019,None,Proposicao,1.00,bill,PL,NaN,NaN
1,CP:2190408,PL 2/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
2,CP:2190417,PL 10/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
3,CP:2190423,PL 15/2019,None,Proposicao,0.00,bill,PL,NaN,NaN
4,CP:2190450,PL 21/2019,None,Proposicao,0.25,bill,PL,NaN,NaN
...,...,...,...,...,...,...,...,...,...
27675,SS:6295,CARLOS HENRIQUE BAQUETA FÁVARO,PSD/MT,Senador,NaN,author,None,PSD,MT
27676,SE:7352398,CPI DA PANDEMIA,None,Orgao,NaN,author,None,NaN,NaN
27677,SE:55226,COMISSÃO DIRETORA,None,Orgao,NaN,author,None,NaN,NaN
27678,SE:3947422,COMISSÃO DE DIREITOS HUMANOS E LEGISLAÇÃO PART...,None,Orgao,NaN,author,None,NaN,NaN


In [52]:
def community_to_df(g, communities, comm_label="comm_idx"):
    rows = []
    for comm_idx, comm in enumerate(communities):
        for v_idx in comm:
            rows.append({'name': g.vs[v_idx]['name'], comm_label: comm_idx})
    return pd.DataFrame(rows).set_index('name', drop=True)

In [75]:
node_leiden_mod_df = node_main_df.join(community_to_df(main_g, comms_leiden_mod, "leiden_mod"), on="name")
node_leiden_mod_df

,name,label,partido,type,score,bigtype,billtype,cod_partido,uf,leiden_mod
0,CP:2187087,PL 5029/2019,None,Proposicao,1.00,bill,PL,NaN,NaN,13
1,CP:2190408,PL 2/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,12
2,CP:2190417,PL 10/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,28
3,CP:2190423,PL 15/2019,None,Proposicao,0.00,bill,PL,NaN,NaN,21
4,CP:2190450,PL 21/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
27675,SS:6295,CARLOS HENRIQUE BAQUETA FÁVARO,PSD/MT,Senador,NaN,author,None,PSD,MT,19
27676,SE:7352398,CPI DA PANDEMIA,None,Orgao,NaN,author,None,NaN,NaN,19
27677,SE:55226,COMISSÃO DIRETORA,None,Orgao,NaN,author,None,NaN,NaN,19
27678,SE:3947422,COMISSÃO DE DIREITOS HUMANOS E LEGISLAÇÃO PART...,None,Orgao,NaN,author,None,NaN,NaN,19


In [57]:
edge_main_df

,from,to,weight
0,CD:160655,CP:538196,1.00
1,CD:141488,CP:559138,1.00
3,CD:160518,CP:601739,0.25
4,CD:151208,CP:614512,1.00
5,CD:73466,CP:946475,1.00
...,...,...,...
49837,SS:6335,SP:8730961,0.25
49838,SS:5502,SP:8730961,0.25
49839,SS:6341,SP:8730961,0.25
49840,SS:6009,SP:8730961,0.25


In [95]:
from pyvis.network import Network
import networkx as nx

def community_grid_positions(G, community_attr="community", cell_size=800, k=0.2, seed=42):
    """Return {node: (x,y)}: spring-layout inside each community, communities on a grid."""
    # map community -> nodes
    comms = {}
    for n, d in G.nodes(data=True):
        c = d.get(community_attr, "NA")
        comms.setdefault(c, []).append(n)

    # grid dims
    C = len(comms)
    cols = int(np.ceil(np.sqrt(C)))
    rows = int(np.ceil(C / cols))
    centers = {}
    idx = 0
    for r in range(rows):
        for c in range(cols):
            if idx >= C: break
            centers[idx] = np.array([c * cell_size, r * cell_size], dtype=float)
            idx += 1

    # positions per community, normalized to a square, then shifted to its center
    pos = {}
    for i, (comm, nodes) in enumerate(comms.items()):
        SG = G.subgraph(nodes)
        # spring layout inside the community
        p = nx.spring_layout(SG, k=k, seed=seed, weight="weight")
        # normalize to [-0.4,0.4] square to avoid overlaps between communities
        P = np.array(list(p.values()))
        if len(P) == 0:
            continue
        mn, mx = P.min(axis=0), P.max(axis=0)
        span = np.where((mx - mn) == 0, 1.0, (mx - mn))
        Pn = (P - mn) / span - 0.5
        Pn *= (cell_size * 0.8)  # size of cluster box
        Pn += centers[i]
        for node, xy in zip(p.keys(), Pn):
            pos[node] = (float(xy[0]), float(xy[1]))
    return pos

In [100]:



G = nx.from_pandas_edgelist(
    df=edge_main_df,
    source="from",
    target="to",
    edge_attr=True,
    create_using=nx.Graph
)

In [101]:
attrs = node_leiden_mod_df.set_index('name', drop=True).to_dict('index')
nx.set_node_attributes(G, attrs)

In [102]:
pos = community_grid_positions(G, "leiden_mod")

In [103]:
net = Network(
    height="700px",
    width="100%",
    notebook=False,
    directed=False,
)


In [105]:
net.set_options('{"physics":{"enabled": false}}')

In [106]:
for n, d in G.nodes(data=True):
    x, y = pos.get(n, (0.0, 0.0))
    net.add_node(
        n,
        label=str(d.get('label', n)),
        group=str(d.get('leiden_mod', 'NA')),
        title=str(d),
        x=x, y=y, physics=False, fixed=True,
    )

In [107]:
for u, v, d in G.edges(data=True):
    net.add_edge(u, v, value=d.get("weight", 1))

In [108]:
net.write_html("leiden_mod.html", notebook=False, open_browser=False)

In [84]:
node_leiden_mod_df

,name,label,partido,type,score,bigtype,billtype,cod_partido,uf,leiden_mod
0,CP:2187087,PL 5029/2019,None,Proposicao,1.00,bill,PL,NaN,NaN,13
1,CP:2190408,PL 2/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,12
2,CP:2190417,PL 10/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,28
3,CP:2190423,PL 15/2019,None,Proposicao,0.00,bill,PL,NaN,NaN,21
4,CP:2190450,PL 21/2019,None,Proposicao,0.25,bill,PL,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
27675,SS:6295,CARLOS HENRIQUE BAQUETA FÁVARO,PSD/MT,Senador,NaN,author,None,PSD,MT,19
27676,SE:7352398,CPI DA PANDEMIA,None,Orgao,NaN,author,None,NaN,NaN,19
27677,SE:55226,COMISSÃO DIRETORA,None,Orgao,NaN,author,None,NaN,NaN,19
27678,SE:3947422,COMISSÃO DE DIREITOS HUMANOS E LEGISLAÇÃO PART...,None,Orgao,NaN,author,None,NaN,NaN,19


In [93]:
def vcs(s):
    return s.value_counts().to_dict()

node_leiden_mod_df.groupby('leiden_mod').agg(
    n_count=('leiden_mod', 'size'),
    billtypes=('billtype', vcs),
    types=('type', vcs),
    partidos=('cod_partido', vcs),
    ufs=('uf', vcs),
    bigtype=('bigtype', vcs),
)

,n_count,billtypes,types,partidos,ufs,bigtype
leiden_mod,,,,,,
0,563,"{'PL': 531, 'PLP': 21, 'PLV': 1, 'PEC': 1}","{'Proposicao': 554, 'Deputado': 9}","{'PSD': 2, 'PP': 2, 'PV': 1, 'REPUBLICANOS': 1...","{'SP': 2, 'BA': 1, 'PE': 1, 'CE': 1, 'RO': 1, ...","{'bill': 554, 'author': 9}"
1,287,"{'PL': 266, 'PLP': 11, 'PLV': 2, 'PEC': 2}","{'Proposicao': 281, 'Deputado': 6}","{'PP': 1, 'MDB': 1, 'DEM': 1, 'PRB': 1, 'PL': ...","{'AL': 1, 'MA': 1, 'PR': 1, 'BA': 1, 'RS': 1, ...","{'bill': 281, 'author': 6}"
2,2474,"{'PL': 2216, 'PLP': 98, 'PEC': 23, 'PLV': 2}","{'Proposicao': 2339, 'Deputado': 135}","{'PL': 60, 'UNIÃO': 17, 'PP': 16, 'MDB': 13, '...","{'SP': 17, 'BA': 11, 'PR': 11, 'RS': 10, 'RJ':...","{'bill': 2339, 'author': 135}"
3,6,{'PL': 5},"{'Proposicao': 5, 'Deputado': 1}",{'PODE': 1},{'SP': 1},"{'bill': 5, 'author': 1}"
4,1894,"{'PL': 1747, 'PLP': 69, 'PEC': 9, 'PLV': 2}","{'Proposicao': 1827, 'Deputado': 67}","{'PT': 64, 'PDT': 1, 'PMDB': 1, 'PV': 1}","{'MG': 10, 'SP': 9, 'BA': 7, 'RS': 5, 'CE': 5,...","{'bill': 1827, 'author': 67}"
...,...,...,...,...,...,...
105,18,{'PL': 17},"{'Proposicao': 17, 'Deputado': 1}",{'PATRIOTA': 1},{'MG': 1},"{'bill': 17, 'author': 1}"
106,16,"{'PL': 14, 'PLP': 1}","{'Proposicao': 15, 'Deputado': 1}",{'AVANTE': 1},{'MG': 1},"{'bill': 15, 'author': 1}"
107,9,{'PL': 8},"{'Proposicao': 8, 'Deputado': 1}",{'PP': 1},{'AL': 1},"{'bill': 8, 'author': 1}"
